# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

---

## Обзор данных

In [1]:
#Импорт библиотек
import pandas as pd
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

In [2]:
#Чтение файла в переменную
try:
    df = pd.read_csv('C:/Users/datasets/data.csv')  # Локальный путь
except:
    df = pd.read_csv('/datasets/data.csv')  # Серверный путь
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#Информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

В данных видны следующе нарушения:
- days_employed — общий трудовой стаж в днях: отрицательные значения, пропуски, тип флоат, некорректные данные.
- education — уровень образования клиента: разный регистр букв
- total_income — ежемесячный доход: тип флоат, пропуски.


**Вывод**

В каждой строке таблицы — данные о клиенте(возраст, пол, количество детей, семейное положение, доход, информация о задолженности и образовании.

Предварительно можно утверждать, что, данных достаточно для оценки способности потенциального заемщика вернуть кредит в срок. 

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Обработка пропусков

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах days_employed и total_income имеется одинаковое количество пропущенных значений.

Попробуем найти связь.


In [5]:
# Вывод строк с пропусками
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


   Из таблицы видна связь межу трудовым стажем и доходом, но, так же видно, наличие образования, детей и работы из чего можно сделать вывод, что данные не заполнены по объективным причинам (например, отсутствие справок, подтверждающие эти данные) или технических збоев. В рамках данного исследования, удалять эти строки считаю нецелесообразным.

In [6]:
# Столбец days_employed в данной работе ненужен, заполним пропуски 0.
# В столбце total_income, пропуски заполним медианой, чтобы избежать влияния сверх высоких или сверх низких зарплат.
df['days_employed']=df['days_employed'].fillna(df['days_employed'].median())
df['total_income']=df['total_income'].fillna(df['total_income'].median())

In [7]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Заполнили пропуски в столбце дохода на медианные значения.
Больше пропусков нет.

### Замена типа данных

In [8]:
#Для удобства восприятия данных, в столбцах days_employed и total_income заменим тип данных на целочисленный.
# Так же возьмем значения по модулю, чтобы избавиться от отрицательных значений.
df['days_employed']=abs(df['days_employed'].astype('int'))
df['total_income']=abs(df['total_income'].astype('int'))
df['debt']=abs(df['debt'].astype('bool'))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: bool(1), int64(6), object(5)
memory usage: 1.8+ MB


**Вывод**

Тип данных в столбце total_income приведен к целочисленному, столбец debt изменен на булеан.



### Обработка дубликатов

In [10]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу


In [11]:
# Приведем текст во всех столбцах к одному виду(маленькие буквы)
df['education']=df['education'].str.lower()
df['income_type']=df['income_type'].str.lower()
df['family_status']=df['family_status'].str.lower()
df['purpose']=df['purpose'].str.lower()

**Проанализируем каждый столбец.**

In [12]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Все ок.

In [13]:
df['dob_years'].value_counts()


35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [14]:
df.loc[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541,0,среднее,1,женат / замужем,0,F,пенсионер,False,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,False,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,False,102166,ремонт жилью
578,0,397856,0,среднее,1,женат / замужем,0,F,пенсионер,False,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,False,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1203,0,среднее,1,женат / замужем,0,F,сотрудник,False,145017,жилье
20462,0,338734,0,среднее,1,женат / замужем,0,F,пенсионер,False,259193,покупка своего жилья
20577,0,331741,0,среднее,1,не женат / не замужем,4,F,пенсионер,False,129788,недвижимость
21179,2,108,0,высшее,0,женат / замужем,0,M,компаньон,False,240702,строительство жилой недвижимости


Ошибки в столбце возраст, есть 0е значения, заменим на средний.

In [15]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean()

In [16]:
df.loc[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [17]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Есть один трансгендер.

In [18]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 и -1 детей, похоже на ошибку из-за человеческого фактора, заменим 20 на 2, -1 на 1.

In [19]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1

In [20]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Ок.

In [21]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Ок.

In [22]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Ок.

In [23]:
#Поиск явных дубликатов
df.duplicated().sum()

71

In [24]:
# Удаление дубликатов со сбросом индексов
df = df.drop_duplicates().reset_index(drop=True) 

**Вывод**

Был проведен анализ столбцов, исправлены значения возраста, количества детей, текст приведен к одному стилю, найдены и удалены дубликаты. 


### Лемматизация

In [25]:
df['purpose'].value_counts().count()

38

В столбце цель кредита, 38 разных целей. Из них можно выделить 4е основных направления.
- автомобиль
- свадьба
- образование
- недвижимость

Напишем функцию, с приминением лемматизации, разбивающую цели на 4е категории:

In [26]:
# Функция получает лемму, соединяет ее в строку и возвращает категорию
def lem(row):
    lemmas =' '.join(m.lemmatize(row))
    if 'автомобиль' in lemmas:
        return 'автомобиль'   
    if 'свадьба' in lemmas:
        return  'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    if 'недвижимость' or 'жил' in lemmas:
        return  'недвижимость'
        

In [27]:
#Применяем функцию к столбцу цели кредита.
df['purpose_group'] = df['purpose'].apply(lem) 
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,свадьба
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,False,255763,покупка жилья,недвижимость
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,False,240525,операции с жильем,недвижимость
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,False,135823,образование,образование
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,False,95856,на проведение свадьбы,свадьба
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,False,144425,покупка жилья для семьи,недвижимость


**Вывод**

При помощи лемматизации, присвоили клиентам категории по цели кредитования.

### Категоризация данных

Разделим клиентов на категории по уровню дохода.

Выделим 3 категории:
- низкий доход
- средний доход
- высокий доход

In [28]:
#Разобьем доход на 3 части по 1/3
df['total_income'].quantile([1/3,2/3])

0.333333    122832.0
0.666667    170833.0
Name: total_income, dtype: float64

In [29]:
# Функция, разбивает клиентов на 3 категории по уровню дохода
def total_income_group(total_income):
    if total_income <=df['total_income'].quantile(0.33):
        return 'низкий доход' 
    if  df['total_income'].quantile(0.33)<total_income< df['total_income'].quantile(0.66):
        return 'средний доход'
    else:
        return 'высокий доход'

In [30]:
# Применяем функцию total_income_group к столбцу с доходом
df['total_income_group']= df['total_income'].apply(total_income_group)      
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,total_income_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,недвижимость,высокий доход
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,автомобиль,низкий доход
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,недвижимость,средний доход
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,образование,высокий доход
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,свадьба,средний доход
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,False,255763,покупка жилья,недвижимость,высокий доход
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,False,240525,операции с жильем,недвижимость,высокий доход
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,False,135823,образование,образование,средний доход
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,False,95856,на проведение свадьбы,свадьба,низкий доход
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,False,144425,покупка жилья для семьи,недвижимость,средний доход


Разделим клиентов на категории по количеству детей.

In [31]:
# Функция, разбивает клиентов на 3 категории по количеству детей.
def children_group(children):
    if children==0:
        return 'бездетный' 
    if  children==1:
        return 'один ребенок'
    else:
        return 'многодетный'

In [41]:
# Применяем функцию children_group к столбцу с детьми
df['children_group']= df['children'].apply(children_group)      
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,total_income_group,children_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,недвижимость,высокий доход,один ребенок
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,автомобиль,низкий доход,один ребенок
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,недвижимость,средний доход,бездетный
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,образование,высокий доход,многодетный
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,свадьба,средний доход,бездетный
5,0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,False,255763,покупка жилья,недвижимость,высокий доход,бездетный
6,0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,False,240525,операции с жильем,недвижимость,высокий доход,бездетный
7,0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,False,135823,образование,образование,средний доход,бездетный
8,2,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,False,95856,на проведение свадьбы,свадьба,низкий доход,многодетный
9,0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,False,144425,покупка жилья для семьи,недвижимость,средний доход,бездетный


**Вывод**

Присвоили клиентам категории по доходу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
#Группируем людей по количеству детей, считаем сумму и количество задолженности, находим конверсию задолженности
children = df.groupby('children_group').agg({'debt': ['count', 'sum']})
children['conversion']=children['debt']['sum'] / children['debt']['count']*100

children

debt         conversion
                count     sum           
children_group                          
бездетный       14091  1063.0   7.543822
многодетный      2508   233.0   9.290271
один ребенок     4855   445.0   9.165808

In [34]:
children_pivot = df.pivot_table(index='children_group', columns='debt',values='children' , aggfunc='count')
children_pivot['conversion'] = children_pivot[1] /(children_pivot[0]+ children_pivot[1])* 100

children_pivot

debt,False,True,conversion
children_group,,,
бездетный,13028,1063,7.543822
многодетный,2275,233,9.290271
один ребенок,4410,445,9.165808


**Вывод**

Лучший показатель возврата у бездетных людей.

Люди с детьми задерживают оплату кредита чаще чем люди без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
family  = df.groupby('family_status').agg({'debt': ['count', 'sum']})
family ['conversion']=family['debt']['sum'] / family['debt']['count']*100

family


debt        conversion
                       count    sum           
family_status                                 
в разводе               1195   85.0   7.112971
вдовец / вдова           959   63.0   6.569343
гражданский брак        4151  388.0   9.347145
женат / замужем        12339  931.0   7.545182
не женат / не замужем   2810  274.0   9.750890

In [36]:
family_pivot = df.pivot_table(index='family_status', columns='debt' , values='children', aggfunc='count')
family_pivot['conversion'] = family_pivot[1] /(family_pivot[0]+ family_pivot[1])* 100

family_pivot

debt,False,True,conversion
family_status,,,
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
не женат / не замужем,2536,274,9.750890


**Вывод**

Лучший показатель у вдов, худший у холостых, у людей в браке средний показатель.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:


total_income  = df.groupby('total_income_group').agg({'debt': ['count', 'sum']})
total_income['conversion']=total_income['debt']['sum'] /total_income['debt']['count']*100

total_income 

debt        conversion
                   count    sum           
total_income_group                        
высокий доход       7295  555.0   7.607951
низкий доход        7080  578.0   8.163842
средний доход       7079  608.0   8.588784

In [38]:
total_income_pivot = df.pivot_table(index='total_income_group', columns='debt', values='children' , aggfunc='count')
total_income_pivot['conversion'] = total_income_pivot[1] /(total_income_pivot[0]+ total_income_pivot[1])* 100

total_income_pivot

debt,False,True,conversion
total_income_group,,,
высокий доход,6740,555,7.607951
низкий доход,6502,578,8.163842
средний доход,6471,608,8.588784


**Вывод**

Люди с высоким доходом выплачивают кредит лучше всех.

- Как разные цели кредита влияют на его возврат в срок?

In [39]:
purpose  = df.groupby('purpose_group').agg({'debt': ['count', 'sum']})
purpose['conversion']=purpose['debt']['sum'] / purpose['debt']['count']*100

purpose


debt        conversion
               count    sum           
purpose_group                         
автомобиль      4306  403.0   9.359034
недвижимость   10811  782.0   7.233373
образование     4013  370.0   9.220035
свадьба         2324  186.0   8.003442

In [40]:
purpose_pivot = df.pivot_table(index='purpose_group', columns='debt', values='children', aggfunc='count')
purpose_pivot['conversion'] = purpose_pivot[1] /(purpose_pivot[0]+ purpose_pivot[1])* 100

purpose_pivot

debt,False,True,conversion
purpose_group,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Кредиты на недвижимость выплачиваются гораздо лучше остальных.

Хуже всего обстоит дело с автокредитами.

##  Общий вывод

Было проведено исследование надёжности заёмщиков. В данных были найдены артефакты, пропуски и ошибки связанные с человеческим фактором, часть из них была устранена, часть не повлияла на результат. Клиенты банка, были разделены на различные категории.

Были выявлены следующие закономерности:
- люди с детьми, чаще имеют задолженности
- люди в категории не 'женат / не замужем' и 'гражданский брак'	 имеют намного больше задолженностей, чем все остальные категории
- люди с высоким доходом имеют меньше задолженностей
- кредиты на недвижимоть погашаются своевременней чем автокредиты.
